purpose: run MAGMA on human control traits (taken from the ext1.0 paper's traits that were least and most significantly associated with externalizing)

# set-up

In [1]:
import pandas as pd
import os
import glob
import statsmodels.stats.multitest

In [2]:
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/gwas_ctrl_hm')

# import control datasets

In [3]:
meta_data=pd.read_csv('ext1_st8_corr_sources_downloaded.csv')

In [16]:
meta_data

,Trait 1,Trait 2,N trait 2,rG,abs(rG),SE,Z,P value,Source,Domain,Subdomain,Phenotype,N,Prepared?,paper,Download,label,ctrl_type
0,Externalizing,Townsend deprivation index*,"112,151",0.707,0.707,0.047,15.167,5.820000e-52,CCACE,Socioeconomic,Economic & Environmental,Neighborhood deprivation (Townsend Index),112151,YES,NaN,http://www.psy.ed.ac.uk/ccace/downloads/Hill_C...,townsend,positive
1,Externalizing,Antisocial behavior,"16,400",0.647,0.647,0.166,3.906,9.400000e-05,CTG Lab,Psychopathology,Psychiatric Symptoms,Antisocial behavior,16400,YES,NaN,http://broadabc.ctglab.nl/documents/p12/BroadA...,antisoc,positive
2,Externalizing,Age of smoking initiation*,"262,990",-0.646,0.646,0.026,-24.708,8.760000e-135,GSCAN,Psychopathology,Substance Use,Age of smoking initiation,262990,YES,NaN,https://conservancy.umn.edu/handle/11299/201564,age_smkinit,positive
3,Externalizing,Friendships satisfaction*,"361,194",-0.001,0.001,0.032,-0.039,9.680000e-01,GWAS Atlas,Socioeconomic,Social,Friendships satisfaction,127735,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.4570.0...,friend_sat,negative
4,Externalizing,Infant birth weight*,"361,194",-0.012,0.012,0.020,-0.609,5.420000e-01,GWAS Atlas,Health & Medicine,Anthropomorphic,Infant birth weight,219088,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.20022....,infant_bw,negative
5,Externalizing,Heart rate*,"361,194",-0.026,0.026,0.018,-1.426,1.540000e-01,GWAS Atlas,Health & Medicine,Cardiometabolic,Heart rate,361411,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.102.0....,hr,negative
6,Externalizing,Age at first facial hair (male)*,"167,020",0.033,0.033,0.024,1.420,1.560000e-01,GWAS Atlas,Demography,Life History & Reproduction,Relative age of first facial hair (male),171805,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.2375.0...,facial_hair,negative
7,Externalizing,Maternal smoking around birth*,"361,194",0.717,0.717,0.021,34.403,2.240000e-259,GWAS Atlas,Socioeconomic,Economic & Environmental,Maternal smoking around birth,331862,YES,NaN,https://atlas.ctglab.nl/ukb2_sumstats/f.1787.0...,maternal_smok,positive
8,Externalizing,Age at menarche*,"194,174",-0.010,0.010,0.021,-0.461,6.450000e-01,ReproGen,Demography,Life History & Reproduction,Age at menarche,252514,YES,NaN,http://www.reprogen.org/Menarche_1KG_NatGen201...,age_menarche,negative
9,Externalizing,LDL,"173,081",0.025,0.025,0.032,0.768,4.430000e-01,GLGC,Health & Medicine,Cardiometabolic,LDL Cholesterol,188577,YES,NaN,http://csg.sph.umich.edu/abecasis/public/lipid...,LDL,negative


In [ ]:
file_dict={'facial_hair':pd.read_csv('age_facial_hair_male.txt.gz',compression='gzip',sep='\t'),
    'age_smkinit':pd.read_csv('age_smkinit.txt.gz',compression='gzip',sep='\t'),
    'antisoc':pd.read_csv('antisoc_behav.tbl',sep='\t',low_memory=False),
    'friend_sat':pd.read_csv('friendship_satisfaction.txt.gz',compression='gzip',sep='\t'),
    'hr':pd.read_csv('heart_rate.txt',sep='\t'),
    'infant_bw':pd.read_csv('infant_birth_weight.txt.gz',compression='gzip',sep='\t'),
    'LDL':pd.read_csv('jointGwasMc_LDL.txt.gz',compression='gzip',sep='\t'),
    'maternal_smok':pd.read_csv('maternal_smok.txt',sep='\t'),
    'townsend':pd.read_csv('townsend_Hill_CB_2016/Hill2016_UKB_Income_summary_results_21112016.txt',sep=' '),
    'age_menarche':pd.read_csv('age_menarche/Menarche_1KG_NatGen2017_WebsiteUpload.txt',delim_whitespace=True),
    'neurot':pd.read_csv('neuroticism/GPC-2.NEUROTICISM.full.txt',sep='\t',header=None),
    'addict-rf':pd.read_csv('hatoum_addiction_euro_2023.txt.gz',sep='\t'),
    'cud':pd.read_csv('CUD_EUR_full_public_11.14.2020.gz',compression='gzip',delim_whitespace=True),
    'adhd':pd.read_csv('adhd.txt',sep='\t'),
    'alc_dep':pd.read_csv('pgc_alcdep.discovery.aug2018_release.txt.gz',delim_whitespace=True),
    'dpw':pd.read_csv('DRINKS_PER_WEEK_GWAS.txt',sep='\t'),
    'risk':pd.read_csv('RISK_GWAS_MA_UKB+replication.txt',sep='\t'),
    'auto_speed':pd.read_csv('AUTOMOBILE_SPEEDING_PROPENSITY_GWAS.txt',sep='\t'),
    'nsex':pd.read_csv('NUMBER_SEXUAL_PARTNERS_GWAS.txt',sep='\t'),
    'bmi':pd.read_csv('bmi.giant-ukbb.meta-analysis.combined.23May2018.txt.gz',compression='gzip',delim_whitespace=True),
    'height':pd.read_csv('Meta-analysis_Wood_et_al+UKBiobank_2018.txt.gz',compression='gzip',sep='\t')
}

# format control files

## addiction risk factor

In [ ]:
file_dict['addict-rf']=file_dict['addict-rf'].dropna()

In [46]:
file_dict['addict-rf']['Chr']=file_dict['addict-rf']['Chr'].apply(lambda s: int(s))
file_dict['addict-rf']['BP']=file_dict['addict-rf']['BP'].apply(lambda s: int(s))

## antisocial

In [6]:
file_dict['antisoc']['CHR']=file_dict['antisoc']['MarkerName'].apply(lambda s: s.split(':')[0])
file_dict['antisoc']['BP']=file_dict['antisoc']['MarkerName'].apply(lambda s: s.split(':')[1])

## neuroticism

In [7]:
file_dict['neurot'].columns=['SNPID','CHR','BP','A1','A2','BETA','SE','PVALUE','NCOH','MAF']
#from the neuroticism read me

## age_menarche

In [8]:
file_dict['age_menarche']['CHR']=file_dict['age_menarche']['Markername'].apply(lambda s: s.split(':')[0][3:])
file_dict['age_menarche']['BP']=file_dict['age_menarche']['Markername'].apply(lambda s: s.split(':')[1])

## LDL

In [9]:
file_dict['LDL']['CHR']=file_dict['LDL']['SNP_hg19'].apply(lambda s: s.split(':')[0][3:])
file_dict['LDL']['BP']=file_dict['LDL']['SNP_hg19'].apply(lambda s: s.split(':')[1])

## alcohol dependence

In [35]:
file_dict['alc_dep']['SNPID']=file_dict['alc_dep']['SNP'].apply(lambda s: s.split(':')[0])

## bmi

In [77]:
file_dict['bmi']['SNP']=file_dict['bmi']['SNP'].apply(lambda s: s.split(':')[0])

In [81]:
file_dict['bmi']=file_dict['bmi'].dropna()

In [93]:
file_dict['bmi']['CHR']=file_dict['bmi']['CHR'].astype(int)

/tmp/ipykernel_404949/2975081436.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_dict['bmi']['CHR']=file_dict['bmi']['CHR'].astype(int)


In [95]:
file_dict['bmi']['POS']=file_dict['bmi']['POS'].astype(int)

/tmp/ipykernel_404949/321537160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_dict['bmi']['POS']=file_dict['bmi']['POS'].astype(int)


## risk

In [123]:
file_dict['risk']=file_dict['risk'].dropna(subset=['MarkerName','CHR','POS','Pval'])

## dpw

In [158]:
len(file_dict['dpw'].dropna(subset=['MarkerName','CHR','POS','Pval']))/len(file_dict['dpw'])

0.9915649552893737

In [159]:
file_dict['dpw']=file_dict['dpw'].dropna(subset=['MarkerName','CHR','POS','Pval'])

## automotive speeding

In [153]:
len(file_dict['auto_speed'].dropna(subset=['MarkerName','CHR','POS','Pval']))/len(file_dict['auto_speed'])

0.9915660851921623

In [154]:
file_dict['auto_speed']=file_dict['auto_speed'].dropna(subset=['MarkerName','CHR','POS','Pval'])

## nsexual partners

In [153]:
len(file_dict['nsex'].dropna(subset=['MarkerName','CHR','POS','Pval']))/len(file_dict['nsex'])

0.9915660851921623

In [161]:
file_dict['nsex']=file_dict['nsex'].dropna(subset=['MarkerName','CHR','POS','Pval'])

# import psychiatric traits

In [5]:
os.chdir('/tscc/projects/ps-palmer/brittany/SUD_cross_species/gwas_hm_psych')

In [18]:
file_dict={
    'anxiety':pd.read_csv('anxiety/ANX_EUR.txt.gz',compression='gzip',delim_whitespace=True, low_memory=False),
    'panic':pd.read_csv('panic_pgc_2019.vcf.gz',compression='gzip',sep='\t',skiprows=72, low_memory=False),
    'asd':pd.read_csv('asd_pgc_2019.gz',compression='gzip',sep='\t', low_memory=False),
    'adhd2022':pd.read_csv('adhd/ADHD2022_iPSYCH_deCODE_PGC.meta.gz',compression='gzip',delim_whitespace=True),
    'scz':pd.read_csv('scz_pgc_2022/PGC3_SCZ_wave3.primary.autosome.public.v3.vcf.tsv.gz',skiprows=73,compression='gzip',sep='\t', low_memory=False),
    'bipolar':pd.read_csv('bipolar_pgc_2024/bip2024_multianc_no23andMe.gz',compression='gzip',delim_whitespace=True),
    'dep':pd.read_csv('mdd_pgc_2025/pgc-mdd2025_no23andMe_div_v3-49-46-01.tsv.gz',compression='gzip',sep='\t',skiprows=54),
    'ptsd':pd.read_csv('ptsd_pgc_2024/trans_ptsd_pcs_v4_aug3_2021.vcf.gz',sep='\t',skiprows=66),
    'ocd':pd.read_csv('ocd_pgc_2024/daner_OCD_full_wo23andMe_190522.gz',sep='\t'),
    'alz':pd.read_csv('alz/Wightman_2021/PGCALZ2sumstatsExcluding23andMe.txt.gz',sep='\t'),
    'park':pd.read_csv('parkinsons_2025_preprint/GP2_euro_ancestry_meta_analysis_2024/GP2_ALL_EUR_ALL_DATASET_HG38_12162024.txt.gz',sep='\t'),
    'als':pd.read_csv('als/GCST90027163_buildGRCh37.tsv.gz',sep='\t'),
    'epilepsy':pd.read_csv('epilepsy/ILAE3_TRANS_all_epilepsy_final.tbl',sep='\t'),
    'anorexia':pd.read_csv('anorexia/pgcAN2.2019-07.vcf.tsv.gz',sep='\t',skiprows=70),
    'park2019':pd.read_csv('GCST009325.h.tsv.gz',compression='gzip',sep='\t')
}

In [23]:
file_dict={
    'bipolar_euro':pd.read_csv('bipolar_pgc_2024/bip2024_eur_no23andMe.gz',compression='gzip',delim_whitespace=True),
    'dep_euro':pd.read_csv('mdd_pgc_2025/pgc-mdd2025_no23andMe_eur_v3-49-24-11.tsv.gz',compression='gzip',sep='\t',skiprows=54),
    'ptsd_euro':pd.read_csv('ptsd_pgc_2024/eur_ptsd_pcs_v4_aug3_2021.vcf.gz',sep='\t',skiprows=66),
    'epilepsy_euro':pd.read_csv('epilepsy/ILAE3_Caucasian_all_epilepsy_final.tbl',delim_whitespace=True),
}

In [34]:
file_dict['epilepsy_euro']=pd.read_csv('epilepsy/ILAE3_Caucasian_all_epilepsy_final.tbl',delim_whitespace=True)

In [1]:
#taken from papers, meta data, etc. may not be perfect but it was the best I could do
sample_size={'anxiety':1096458,
'panic':2147+7760,
'asd':8382+27969,
'adhd2022':38691+186843,
'scz':71554+97863,
'bipolar':2954535-2100833,
'dep':537363+2061567,
'ptsd':150793+1130140,
'ocd':23493+1114613,
'alz':1126563,
'park':63555+17700+1746386,
'als':138086,
'epilepsy':29944+52538,
'anorexia':16992+55525,
'park2019':37688+18618+1411006-2448-571411,
'dep_euro':1152656,
'ptsd_euro':137136+1085746,
'bipolar_euro':4068461,
'epilepsy_euro':82482+69995}


# format psych trait data

In [26]:
file_dict['alz']['ID']=file_dict['alz'][' chr'].astype(str)+'_'+file_dict['alz']['PosGRCh37'].astype(str)

KeyError: 'alz'

In [ ]:
file_dict['alz']=file_dict['alz'].dropna()

In [ ]:
file_dict['park']=file_dict['park'].dropna()

In [16]:
'''file_dict['park2019']=file_dict['park2019'].dropna()
file_dict['park2019'][['CHR','POS']]=file_dict['park2019']['SNP'].str.split(":",expand=True)
file_dict['park2019']['CHR']=file_dict['park2019']['CHR'].str.replace('chr','')'''

In [22]:
file_dict['park2019']

,chromosome,base_pair_location,effect_allele,other_allele,beta,standard_error,effect_allele_frequency,p_value,rsid,N_cases,N_controls,hm_coordinate_conversion,hm_code,variant_id
0,1,15893,C,T,0.0287,0.0840,0.8484,0.7327,rs555382915,1169,968,lo,13,1_15893_T_C
1,1,54591,G,A,0.6797,0.8869,0.0062,0.4435,rs561234294,527,472,lo,11,1_54591_A_G
2,1,54676,T,C,-0.0996,0.0656,0.3166,0.1287,rs2462492,1169,968,lo,10,1_54676_C_T
3,1,79188,T,G,0.6704,0.8795,0.0062,0.4459,rs534350410,527,472,lo,10,1_79188_G_T
4,1,82103,C,T,0.0282,0.1632,0.9642,0.8630,rs2020400,1169,968,lo,13,1_82103_T_C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17443089,22,50795735,A,G,-1.5364,1.7089,0.0014,0.3686,rs192982090,769,195,lo,10,22_50795735_G_A
17443090,22,50795771,C,T,0.1478,1.9246,0.0020,0.9388,rs6010092,512,282,lo,11,22_50795771_T_C
17443091,22,50798635,C,T,0.0108,0.0457,0.2937,0.8132,rs3896457,6634,4884,lo,11,22_50798635_T_C
17443092,22,50800085,G,C,-1.8077,1.9812,0.0011,0.3615,rs1181188060,476,462,lo,6,22_50800085_C_G


# export magma input files

In [10]:
for f in file_dict.keys():
    print(f)
    print(file_dict[f].columns)

park2019
Index(['SNP', 'A1', 'A2', 'freq', 'b', 'se', 'p', 'N_cases', 'N_controls'], dtype='object')


In [6]:
colname_dict={
    'facial_hair':['SNP','CHR','BP','P','A1','A2'],
    'age_smkinit':['RSID','CHROM','POS','PVALUE'],
    'antisoc':['MarkerName','CHR','BP','P-value','Allele1','Allele2'],
    'friend_sat':['SNP','CHR','BP','P','A1','A2'], 
    'hr':['SNP','CHR','BP','P','A1','A2'],
    'infant_bw':['SNP','CHR','BP','P','A1','A2'],
    'LDL':['rsid','CHR','BP','P-value'], 
    'maternal_smok':['SNP','CHR','BP','P','A1','A2'],
    'townsend':['Markername','Chromosome','Position','P-value'],
    'age_menarche':['Markername','CHR','BP','Pvalue','Allele1','Allele2'], 
    'neurot':['SNPID','CHR','BP','PVALUE'],
    'addict-rf':['SNP','Chr','BP','P','A1','A2'],
    'cud':['SNP','CHR','BP','P','A1','A2'],
    'adhd2022':['SNP','CHR','BP','P','A1','A2'],
    'alc_dep':['SNPID','CHR','BP','P','A1','A2'],
    'dpw':['MarkerName','CHR','POS','Pval','A1','A2'],
    'risk':['MarkerName','CHR','POS','Pval','A1','A2'],
    'auto_speed':['MarkerName','CHR','POS','Pval','A1','A2'],
    'nsex':['MarkerName','CHR','POS','Pval','A1','A2'],
    'bmi':['SNP','CHR','POS','P','Tested_Allele','Other_Allele'],
    'height':['SNP','CHR','POS','P','Tested_Allele','Other_Allele']
}

In [37]:
for x in file_dict.keys():
    print (x)
    display(file_dict[x].head())

bipolar_euro


,SNP,CHR,BP,A1,A2,INFO,OR,SE,P,ngt,Direction,HetISqt,HetDf,HetPVa,Nca,Nco,Neff_half,HRC_FRQ_A1
0,rs10,7,92383888,A,C,0.812,1.01045,0.0175,0.5538,1,---+++--+++-++-+-++-??-++++-+------+--?+++++++...,0.0,61,0.79660,52947,719963,71385.55,0.052998
1,rs1000000,12,126890980,G,A,0.975,1.00190,0.0089,0.8348,23,---++-+-++------+--++-++-+-+++-++++---+-----+-...,2.8,67,0.41300,59287,781022,81683.64,0.777381
2,rs10000003,4,57561647,A,G,0.976,1.01187,0.0080,0.1427,1,---+++--+++-+++-+++-+----+-++-+---+-++++--+--+...,0.0,67,0.59140,59287,781022,81683.64,0.300071
3,rs10000005,4,85161558,G,A,0.964,0.99810,0.0075,0.8025,1,+-+-----+++-+---+++++--+---++---+---+----+++++...,19.0,66,0.09431,58452,778655,80449.13,0.457630
4,rs10000006,4,108826383,T,C,0.909,0.96967,0.0227,0.1737,2,-++-+?+-+--++----+++??-+-+------?-+-+-++--+-++...,2.1,59,0.42920,49495,503352,64067.76,0.961533


dep_euro


,#CHROM,POS,ID,EA,NEA,BETA,SE,PVAL,FCAS,FCON,IMPINFO,NEFF,NCAS,NCON,HETI,HETDF,HETPVAL
0,1,752566,rs3094315,G,A,0.002796,0.0047,0.5535,0.177,0.179,0.843,927713.52,341197,989052,39.1,12,0.07303
1,1,752721,rs3131972,A,G,0.002696,0.0048,0.5773,0.180,0.181,0.829,934966.92,343527,1004757,33.4,15,0.09513
2,1,753541,rs2073813,A,G,0.002696,0.0048,0.5712,0.147,0.150,0.880,1014821.72,364797,1341816,27.5,17,0.13490
3,1,754182,rs3131969,A,G,0.002796,0.0048,0.5677,0.148,0.150,0.882,1014821.72,364797,1341816,24.8,17,0.16270
4,1,754192,rs3131968,A,G,0.002796,0.0048,0.5680,0.148,0.150,0.882,1014821.72,364797,1341816,24.6,17,0.16490


ptsd_euro


,#CHROM,ID,POS,A1,A2,FREQ,NEFF,Z,P,DIRE
0,2,rs6759922,22450249,A,G,0.4573,638463.0,8.821,1.134000e-18,+++-+--+--+++++-++-++++----++-+-++-+-+++++++++...
1,11,rs488769,57450720,A,C,0.6558,641533.0,-8.815,1.198000e-18,+-+-++--++-----++++-+++++---++++-----+--++-+--...
2,7,rs1476535,114071035,T,C,0.5607,529716.0,8.807,1.284000e-18,++--++--+-++++-++--+++++-+-+-+++++++++-++++--+...
3,7,rs10228494,114083550,C,G,0.4362,638285.0,-8.740,2.333000e-18,--+---++-+----+--++-----+-+-+---------+----++-...
4,2,rs13387644,22448443,A,G,0.4581,638463.0,8.739,2.357000e-18,+++-+--+--+++++-++-++++----++-+-++-+-+++++++++...


epilepsy_euro


,CHR,BP,MarkerName,Allele1,Allele2,Freq1,FreqSE,Effective_N,Z-score,P-value,Direction,Beta,SE
0,6,130840091,rs2326918,a,g,0.8493,0.0066,64549.2,-1.321,0.1866,--+---+??,-0.010277,0.007779
1,7,145771806,rs6977693,t,c,0.8541,0.0040,63985.7,-1.294,0.1956,----+-+??,-0.010247,0.007919
2,11,100009976,rs12364336,a,g,0.8720,0.0020,64942.1,0.185,0.8532,-----++??,0.001536,0.008305
3,1,166367755,rs12562373,a,g,0.7572,0.0050,62937.9,1.379,0.1680,+-?--++??,0.009065,0.006573
4,14,86737556,rs2135099,a,g,0.1772,0.0092,65083.7,0.099,0.9208,-++-++-??,0.000719,0.007259


In [41]:
colname_dict={
    'anxiety':['SNP','CHR','BP','P','A1','A2'],
    'panic':['ID','#CHROM','POS','PVAL','A1','A2'],
    'asd':['SNP','CHR','BP','P','A1','A2'],
    'adhd2022':['SNP','CHR','BP','P','A1','A2'],
    'scz':['ID','CHROM','POS','PVAL','A1','A2'],
    'bipolar':['SNP','CHR','BP','P','A1','A2'],
    'dep':['ID','#CHROM','POS','PVAL','EA','NEA'],
    'ptsd':['ID','#CHROM','POS','P','A1','A2'],
    'ocd':['SNP','CHR','BP','P','A1','A2'],
    'alz':['ID',' chr','PosGRCh37','p','testedAllele','otherAllele'],
    'park':['SNP_ID','chromosome','base_pair_position','p_value','effect_allele','other_allele'],
    'als':['rsid','chromosome','base_pair_location','p_value','effect_allele','other_allele'],
    'epilepsy':[' MarkerName ','CHR ',' BP ',' P-value ',' Allele1 ',' Allele2 '],
    'anorexia':['ID','CHROM','POS','PVAL','ALT','REF'],
    'park2019':['rsid','chromosome','base_pair_location','p_value','effect_allele','other_allele'],
    'bipolar_euro':['SNP','CHR','BP','P','A1','A2'],
    'dep_euro':['ID','#CHROM','POS','PVAL','EA','NEA'],
    'epilepsy_euro':['MarkerName','CHR','BP','P-value','Allele1','Allele2'],
    'ptsd_euro':['ID','#CHROM','POS','P','A1','A2']

}

In [46]:
def ex_magma(snp,colname,N,prefix,dir,use_RSID=False):
    #purpose- format pvalue and pos files- specifics for mlma
    if use_RSID==False:
        pos=snp[[colname[0],colname[1],colname[2]]]
    else:
        pos=snp[['RSID',colname[1],colname[2]]]
    pos.columns=['SNP','CHR','POS']
    pos=pos.dropna()
    #pos['CHR']=pos['CHR'].astype(int)
    pos['POS']=pos['POS'].astype(int)
    pos.to_csv(dir+prefix+'_pos.tsv',sep='\t',index=False)
    print(pos.head())
    print('pos file written to '+dir+prefix+'_pos.tsv')
    if use_RSID==False:
        pval=snp[[colname[0],colname[3]]]
    else:
        pval=snp[['RSID',colname[3]]]
    pval.columns=['SNP','P']
    pval=pval.dropna()
    #pval['P']=pval['P'].astype(float)
    #pval['N']=N
    print(N)
    pval.insert(2, 'N', N)
    print(pval.head())

    pval.to_csv(dir+prefix+'_pval.tsv',sep='\t',index=False)
    print('pval file written to '+dir+prefix+'_pval.tsv')
    return pos,pval

In [44]:
def ex_magma_comb(snp,colname,N,prefix,dir):
    #purpose- format pvalue and pos files- specifics for mlma
    t=snp[[colname[0],colname[1],colname[2],colname[3]]]

    t.columns=['SNP','CHR','POS','P']
    t=t.dropna()
    t['CHR']=t['CHR'].astype(int)
    t['POS']=t['POS'].astype(int)
    t['P']=t['P'].astype(float)
    t.insert(4, 'N', N)
    print(t.head())

    t.to_csv(dir+prefix+'_comb.tsv',sep='\t',index=False)
    print('combined file written to '+dir+prefix+'_comb.tsv')
    return t

In [48]:
# for phenotypes in psychiatric set
#ls=file_dict.keys()
ls=['epilepsy_euro']
for t in ls:
    print(t)
    pos,pval=ex_magma(file_dict[t],colname_dict[t],int(sample_size[t]),t,'magma/')

epilepsy_euro
          SNP  CHR        POS
0   rs2326918    6  130840091
1   rs6977693    7  145771806
2  rs12364336   11  100009976
3  rs12562373    1  166367755
4   rs2135099   14   86737556
pos file written to magma/epilepsy_euro_pos.tsv
152477
          SNP       P       N
0   rs2326918  0.1866  152477
1   rs6977693  0.1956  152477
2  rs12364336  0.8532  152477
3  rs12562373  0.1680  152477
4   rs2135099  0.9208  152477
pval file written to magma/epilepsy_euro_pval.tsv


# check output files for failed jobs

In [5]:
print(glob.glob("magma/*.annot"))

['magma/infant_bw.genes.annot', 'magma/hr.genes.annot', 'magma/alc_dep.genes.annot', 'magma/antisoc.genes.annot', 'magma/facial_hair.genes.annot', 'magma/addict-rf.genes.annot', 'magma/age_menarche.genes.annot', 'magma/neurot.genes.annot', 'magma/adhd.genes.annot', 'magma/bmi.genes.annot', 'magma/townsend.genes.annot', 'magma/nsex.genes.annot', 'magma/friend_sat.genes.annot', 'magma/height.genes.annot', 'magma/auto_speed.genes.annot', 'magma/cud.genes.annot', 'magma/dpw.genes.annot', 'magma/risk.genes.annot', 'magma/LDL.genes.annot', 'magma/age_smkinit.genes.annot', 'magma/maternal_smok.genes.annot']


In [6]:
#traits where job succeeded
list(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.out")))

['infant_bw',
 'age_menarche',
 'neurot',
 'age_smkinit',
 'friend_sat',
 'nsex',
 'bmi',
 'hr',
 'risk',
 'auto_speed',
 'dpw',
 'maternal_smok',
 'adhd',
 'antisoc',
 'cud',
 'height',
 'LDL',
 'addict-rf',
 'townsend',
 'alc_dep',
 'facial_hair']

In [7]:
#traits where job failed
rerun=set(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.annot") ))-set(list(map((lambda x: x.split('/')[1].split('.')[0]),glob.glob("magma/*.out") )))
print(rerun)

set()


all initally failed jobs don't have rsids- required formatting apparently, for use of 1000 genomes. 
none have CHR X and Y- all are numerically ordered

only antisoc failed- there is a pvalue that isn't parsable

# merge data dictionary for traits without SNPs with SNP dictionary to add RSIDs

In [39]:
#use this one
#downloaded from https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh38p7/VCF/
# on 30 may 2025
snp_ref=pd.read_csv('~/bsl/magma_v1/NCBI38/00-common_all.vcf.gz',compression='gzip',delim_whitespace=True,skiprows=56)

/scratch/bsleger/job_5736024/ipykernel_4097341/3794925557.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  snp_ref=pd.read_csv('~/bsl/magma_v1/NCBI38/00-common_all.vcf.gz',compression='gzip',delim_whitespace=True,skiprows=56)


In [40]:
snp_ref['marker_merge']=(snp_ref['#CHROM']).map(lambda x:str(x))+':'+(snp_ref['POS']).map(lambda x:str(x))

## add RSIDs to alz

In [7]:
import pandas as pd

In [8]:
# use this one if using old version of human genome (ie with ALZ)
# downloaded from here https://ftp.ncbi.nih.gov/snp/organisms/human_9606_b151_GRCh37p13/VCF/
# on 30 may 2025
snp_ref_old=pd.read_csv('~/bsl/magma_v1/NCBI37/00-common_all.vcf.gz',compression='gzip',delim_whitespace=True,skiprows=56)

/scratch/bsleger/job_5736024/ipykernel_2075038/771078087.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  snp_ref_old=pd.read_csv('~/bsl/magma_v1/NCBI37/00-common_all.vcf.gz',compression='gzip',delim_whitespace=True,skiprows=56)


In [9]:
snp_ref_old['marker_merge']=(snp_ref_old['#CHROM']).map(lambda x:str(x))+':'+(snp_ref_old['POS']).map(lambda x:str(x))

In [10]:
snp_ref_old=snp_ref_old[['marker_merge','ID']]

In [11]:
t='park2019'

In [18]:
tbl=file_dict[t][colname_dict[t]]

tbl.columns=['SNP','CHR','POS','P','A1','A2']
tbl=tbl.dropna()
tbl['CHR']=tbl['CHR'].astype(int)
tbl['POS']=tbl['POS'].astype(int)
tbl['P']=tbl['P'].astype(float)
tbl.insert(4, 'N', sample_size[t])
print(tbl.head())
tbl['marker_merge']=(tbl['CHR']).map(lambda x:str(x))+':'+(tbl['POS']).map(lambda x:str(x))

              SNP  CHR        POS       P       N A1 A2
0  chr11:88249377   11   88249377  0.3771  893453  T  C
1   chr1:60320992    1   60320992  0.1846  893453  A  G
2   chr2:18069070    2   18069070  0.6163  893453  T  C
3  chr8:135908647    8  135908647  0.1887  893453  A  G
4   chr12:3871714   12    3871714  0.8886  893453  A  C


In [49]:
file_dict.keys()

dict_keys(['bipolar_euro', 'dep_euro', 'ptsd_euro', 'epilepsy_euro'])

In [20]:
len(set(tbl['marker_merge']).intersection(snp_ref_old['marker_merge']))/len(set(tbl['marker_merge']))

0.7411554372984116

In [28]:
tbl=tbl.merge(snp_ref_old[['marker_merge','ID']],on='marker_merge',how='inner')

In [29]:
tbl=tbl[['ID','CHR','POS','P','N']]
tbl.columns=[['SNP','CHR','POS','P','N']]

In [30]:
tbl.to_csv('magma/'+t+'_comb.tsv',sep='\t',index=False)

## add RSIDs to park

In [41]:
t='park2019'

In [48]:
tbl=file_dict[t][colname_dict[t]]

tbl.columns=['SNP','CHR','POS','P','A1','A2']
tbl=tbl.dropna()
tbl['CHR']=tbl['CHR'].astype(int)
tbl['POS']=tbl['POS'].astype(int)
tbl['P']=tbl['P'].astype(float)
tbl.insert(4, 'N', sample_size[t])
print(tbl.head())
tbl['marker_merge']=(tbl['CHR']).map(lambda x:str(x))+':'+(tbl['POS']).map(lambda x:str(x))

              SNP  CHR        POS       P       N A1 A2
0  chr11:88249377   11   88249377  0.3771  893453  T  C
1   chr1:60320992    1   60320992  0.1846  893453  A  G
2   chr2:18069070    2   18069070  0.6163  893453  T  C
3  chr8:135908647    8  135908647  0.1887  893453  A  G
4   chr12:3871714   12    3871714  0.8886  893453  A  C


In [55]:
len(set(tbl['marker_merge']).intersection(snp_ref['marker_merge']))/len(set(tbl['marker_merge']))

KeyError: 'marker_merge'

In [50]:
tbl=tbl.merge(snp_ref[['marker_merge','ID']],on='marker_merge',how='inner')

In [51]:
tbl=tbl[['ID','CHR','POS','P','N']]
tbl.columns=[['SNP','CHR','POS','P','N']]

In [122]:
tbl.to_csv('magma/'+t+'_comb.tsv',sep='\t',index=False)

In [55]:
tbl[['CHR','POS','P','N']].to_csv('magma/'+t+'_comb.tsv',sep='\t',index=False)

## fix antisocial error

In [ ]:
t='antisoc'

In [ ]:
file_dict[t]['marker_merge']=file_dict[t][colname_dict[t][1]].map(lambda x:str(x))+':'+file_dict[t][colname_dict[t][2]].map(lambda x:str(x))+':'+file_dict[t][colname_dict[t][4]].str.upper()+':'+file_dict[t][colname_dict[t][5]].str.upper()
file_dict[t]=file_dict[t].merge(snp[['RSID','marker_merge']],on='marker_merge',how='inner')

,MarkerName,Allele1,Allele2,Weight,Zscore,P-value,Direction,CHR,BP,marker_merge,RSID
1283482,14:22262789,t,g,16400.0,0.9190,0.3583,++--+,14,22262789,14:22262789:T:G,rs12886640
1283483,6:50483161,a,c,16400.0,-0.2930,0.7693,--+-+,6,50483161,6:50483161:A:C,rs79400560
1283484,2:134909570,t,c,1379.0,0.6938,--+++,NaN,2,134909570,2:134909570:T:C,rs113396222
1283485,11:126026645,t,c,16400.0,-0.4780,0.6326,-+-+-,11,126026645,11:126026645:T:C,rs12421177


In [62]:
#remove line with missing pvalue- and remove any others that may have non-numeric p-values
file_dict[t]['P-value']=pd.to_numeric(file_dict[t]['P-value'],'coerce')
file_dict[t]=file_dict[t].dropna(subset=['P-value'])

In [64]:
file_dict[t].loc[1283482:1283485] # check line 1283485- one that threw the error- it's missing a pvalue

,MarkerName,Allele1,Allele2,Weight,Zscore,P-value,Direction,CHR,BP,marker_merge,RSID
1283482,14:22262789,t,g,16400.0,0.919,0.3583,++--+,14,22262789,14:22262789:T:G,rs12886640
1283483,6:50483161,a,c,16400.0,-0.293,0.7693,--+-+,6,50483161,6:50483161:A:C,rs79400560
1283485,11:126026645,t,c,16400.0,-0.478,0.6326,-+-+-,11,126026645,11:126026645:T:C,rs12421177


In [65]:
pos,pval=ex_magma(file_dict[t],colname_dict[t],int(meta_data[meta_data['label']==t]['N']),t,'magma/',True)

           SNP CHR        POS
0     rs667647   5   29439275
1  rs113534962   5   85928892
2    rs2366866  10  128341232
3     rs472303   3   62707519
4   rs11725240   4   55643311
pos file written to magma/antisoc_pos.tsv
16400
       N          SNP       P
0  16400     rs667647  0.1959
1  16400  rs113534962  0.9342
2  16400    rs2366866  0.3463
3  16400     rs472303  0.6594
4  16400   rs11725240  0.8004
pval file written to magma/antisoc_pval.tsv


# merge in gene names

In [50]:
gene_loc=pd.read_csv('/tscc/projects/ps-palmer/brittany/magma_v1/NCBI38/NCBI38.gene.loc',sep='\t',header=None)
gene_loc.columns=['ID','CHR','START','STOP','STAND','GENE_NAME']

In [51]:
file_ls=file_dict.keys()

In [15]:
file_ls=['cud', 'adhd2022', 'alc_dep', 'dpw', 'risk', 'auto_speed', 'nsex', 'bmi', 'height']

In [52]:
magma_dict={}
for t in file_ls:
    magma_dict[t]=pd.read_csv('magma/'+t+'.genes.out',delim_whitespace=True)
    magma_dict[t]=gene_loc[['ID','GENE_NAME']].merge(magma_dict[t], left_on='ID',right_on='GENE',how='right').reset_index().drop(['index','GENE'],axis=1)
    magma_dict[t].columns=['ID', 'GENE', 'CHR', 'START', 'STOP', 'NSNPS', 'NPARAM', 'N','ZSTAT', 'P']
    magma_dict[t]['Q']=statsmodels.stats.multitest.fdrcorrection(magma_dict[t]['P'],alpha=0.05,method='indep',is_sorted=False)[1]
    magma_dict[t].to_csv('magma/seed_genes/'+t+'_annot.tsv',sep='\t',index=False)

In [ ]:
# plot manhattan plots